In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secret import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'FB'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)
api_url


{'avgTotalVolume': 28038966, 'calculationPrice': 'close', 'change': 0.99, 'changePercent': 0.00506, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Meta Platforms Inc - Class A', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': '15 minute delayed price', 'highTime': 1654718399999, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 196.64, 'iexCloseTime': 1654718399532, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 194.47, 'iexOpenTime': 1654695000305, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1654718399899, 'latestPrice': 196.64, 'latestSource': 'Close', 'latestTime': 'June 8, 2022', 'latestUpdate': 1654718400792, 'latestVolume': None, 'low': None, 'lowSource': '15 minute delayed price', 'lowTime': 165469500

'https://cloud.iexapis.com/stable/stock/FB/quote?token=sk_7e78404b4dbf413c85b2f591eb48c050'

In [5]:
price = data['latestPrice']
print(price)
market_cap = data['marketCap']
market_cap

196.64


561210585170

In [6]:
my_columns = ['Ticker','Stock Price','Market Capitalizaton','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalizaton,Number of Shares to Buy


In [7]:
final_dataframe.append(
   pd.Series(
   [
       symbol,
       price,
       market_cap,
       'N/A'
   ],
    index = my_columns
   ),
    ignore_index = True
)

C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\1244930420.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalizaton,Number of Shares to Buy
0,FB,196.64,561210585170,N/A


In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns), 
    ignore_index = True
    )

C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\1200916656.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\1200916656.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\1200916656.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\1200916656.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_da

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalizaton,Number of Shares to Buy
0,A,156.92,46459624517,N/A
1,AAL,17.02,11061313097,N/A
2,AAP,151.03,9079657636,N/A
3,AAPL,134.76,2134088063880,N/A
4,ABBV,153.6,271638606029,N/A


In [10]:
def chunks(lst,n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url=f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns), 
            ignore_index = True
        )
        
final_dataframe        

C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\4047315075.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\4047315075.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\4047315075.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\4047315075.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = fina

C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\4047315075.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\4047315075.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\4047315075.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Aman Vishwakarma\AppData\Local\Temp\ipykernel_4512\4047315075.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = fina

,Ticker,Stock Price,Market Capitalizaton,Number of Shares to Buy
0,A,156.92,46459624517,N/A
1,AAL,17.02,11061313097,N/A
2,AAP,151.03,9079657636,N/A
3,AAPL,134.76,2134088063880,N/A
4,ABBV,153.6,271638606029,N/A
...,...,...,...,...
95,CINF,108.75,17093793495,N/A
96,CL,77.85,65021426248,N/A
97,CLX,146.11,18027752690,N/A
98,CMA,68.54,8975478798,N/A


In [12]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:10000000


In [13]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe
# number_of_apple_shares = position_size/134.74
# print(math.floor(number_of_apple_shares))

,Ticker,Stock Price,Market Capitalizaton,Number of Shares to Buy
0,A,156.92,46459624517,637
1,AAL,17.02,11061313097,5875
2,AAP,151.03,9079657636,662
3,AAPL,134.76,2134088063880,742
4,ABBV,153.6,271638606029,651
...,...,...,...,...
95,CINF,108.75,17093793495,919
96,CL,77.85,65021426248,1284
97,CLX,146.11,18027752690,684
98,CMA,68.54,8975478798,1459


In [14]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine= 'xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades', index = False)

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
     {
         'font_color':font_color,
         'bg_color': background_color,
         'border': 1
     }
)

dollar_format = writer.book.add_format(
     {
         'num_format': '$0.00',
         'font_color':font_color,
         'bg_color': background_color,
         'border': 1
     }
)

integer_format = writer.book.add_format(
     {
         'num_format': '0',
         'font_color':font_color,
         'bg_color': background_color,
         'border': 1
     }
)

In [16]:
# writer.sheets['Recommended Trades'].set_column('A:A' , 18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B' , 18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C' , 18,string_format)
# writer.sheets['Recommended Trades'].set_column('D:D' , 18,string_format)
# writer.save()

writer.sheets['Recommended Trades'].write('A1','Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy', integer_format)

0

In [17]:
column_format ={
    'A':['Ticker',string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number Of Shares to Buy', integer_format],
}

for column in column_format.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}' , 18,column_format[column][1])

In [18]:
writer.save()